In [53]:
import pandas as pd

In [54]:
dest = pd.read_excel(r"C:\Users\jacob\OneDrive\Desktop\Fall 2024\Telling Stories\housing_report\report-data-2024\Destinations_10.29.22-10.29.2024.xlsx")


In [55]:
# Creating the repeat dataframe
repeat = dest[dest["HMIS ID"].duplicated(keep=False)]

# Display the resulting dataframe
print(repeat)

      HMIS ID  \
4      179138   
6      162304   
19     191061   
25     184917   
33     168173   
...       ...   
1892   171881   
1893   170175   
1900   181072   
1902   162067   
1909   185363   

                                                                                            Entry/Exit Destination  \
4                                                                                                   Deceased (HUD)   
6                                                                                                   Deceased (HUD)   
19                                                                                                  Deceased (HUD)   
25                                                                                                  Deceased (HUD)   
33    Emergency shelter, including hotel or motel paid for with emergency shelter voucher, Host Home shelter (HUD)   
...                                                                                    

In [56]:
# Sorting the repeat dataframe by the "HMIS ID" column
repeat_sorted = repeat.sort_values(by="HMIS ID")

# Display the sorted dataframe
print(repeat_sorted)


      HMIS ID  \
88      28351   
75      28351   
1447   110591   
1448   110591   
109    114968   
...       ...   
1539   202261   
550    202778   
551    202778   
1557   205388   
105    205388   

                                               Entry/Exit Destination  \
88                 Jail, prison, or juvenile detention facility (HUD)   
75    Hotel or motel paid for without emergency shelter voucher (HUD)   
1447         Psychiatric hospital or other psychiatric facility (HUD)   
1448         Psychiatric hospital or other psychiatric facility (HUD)   
109                Jail, prison, or juvenile detention facility (HUD)   
...                                                               ...   
1539               Rental by client, no ongoing housing subsidy (HUD)   
550                                 No exit interview completed (HUD)   
551                                 No exit interview completed (HUD)   
1557               Rental by client, no ongoing housing subsidy (

In [57]:
print(repeat.columns)


Index(['HMIS ID', 'Entry/Exit Destination', 'Entry Date', 'Exit Date',
       'Days to Destination', 'Veteran', 'Date of Birth', 'Age',
       'Household Type', 'Pregnant', 'Chronicity', 'Race (Retired)',
       'Ethnicity (Retired)'],
      dtype='object')


In [58]:
# Count the occurrences of each "HMIS ID"
repeat_counts = repeat["HMIS ID"].value_counts()

# Get the maximum count (highest number of repeats)
max_repeats = repeat_counts.max()

print(f"The highest number of repeating values for 'HMIS ID' is: {max_repeats}")


The highest number of repeating values for 'HMIS ID' is: 4


In [ ]:
# Define the aggregation function with the new columns
def process_group(group):
    # Sort by Exit Date
    sorted_group = group.sort_values(by="Exit Date")
    
    # Extract entry, exit dates, and destinations
    entry_dates = sorted_group["Entry Date"].tolist()
    exit_dates = sorted_group["Exit Date"].tolist()
    destinations = sorted_group["Entry/Exit Destination"].tolist()
    
    return pd.Series({
        "hmis_id": group["HMIS ID"].iloc[0],
        "repeats": len(group),
        "entry_1": entry_dates[0] if len(entry_dates) > 0 else None,
        "exit_1": exit_dates[0] if len(exit_dates) > 0 else None,
        "entry_2": entry_dates[1] if len(entry_dates) > 1 else None,
        "exit_2": exit_dates[1] if len(exit_dates) > 1 else None,
        "entry_3": entry_dates[2] if len(entry_dates) > 2 else None,
        "exit_3": exit_dates[2] if len(exit_dates) > 2 else None,
        "entry_4": entry_dates[3] if len(entry_dates) > 3 else None,
        "exit_4": exit_dates[3] if len(exit_dates) > 3 else None,
        "des_1": destinations[0] if len(destinations) > 0 else None,
        "des_2": destinations[1] if len(destinations) > 1 else None,
        "des_3": destinations[2] if len(destinations) > 2 else None,
        "des_4": destinations[3] if len(destinations) > 3 else None,
        "veteran": group["Veteran"].mode().iloc[0] if not group["Veteran"].mode().empty else None,
        "d_o_b": group["Date of Birth"].mode().iloc[0] if not group["Date of Birth"].mode().empty else None,
        "household": group["Household Type"].mode().iloc[0] if not group["Household Type"].mode().empty else None,
        "pregnant": sum(group["Pregnant"] != "No"),
        "chronic": group["Chronicity"].mode().iloc[0] if not group["Chronicity"].mode().empty else None,
        "race": group["Race (Retired)"].mode().iloc[0] if not group["Race (Retired)"].mode().empty else None,
        "ethn": group["Ethnicity (Retired)"].mode().iloc[0] if not group["Ethnicity (Retired)"].mode().empty else None,
        "age" : group["Age"].mode().iloc[0] if not group["Age"].mode().empty else None
    })

# Group by "HMIS ID" and apply the aggregation function
rep = repeat.groupby("HMIS ID").apply(process_group).reset_index(drop=True)

# Display the resulting dataframe
print(rep)


     hmis_id  repeats    entry_1     exit_1    entry_2     exit_2 entry_3  \
0      28351        2 2021-03-30 2022-12-19 2023-07-01 2024-01-13     NaT   
1     110591        2 2023-02-22 2023-03-17 2024-01-29 2024-07-06     NaT   
2     114968        2 2022-04-29 2023-01-17 2023-05-16 2023-09-01     NaT   
3     117893        2 2023-02-10 2023-07-03 2024-04-08 2024-09-03     NaT   
4     132248        2 2022-09-23 2022-12-05 2023-07-05 2023-07-06     NaT   
..       ...      ...        ...        ...        ...        ...     ...   
161   201293        2 2023-08-31 2023-09-15 2023-09-15 2024-01-05     NaT   
162   201294        2 2023-08-31 2023-09-15 2023-09-15 2024-01-05     NaT   
163   202261        2 2023-10-12 2024-02-02 2024-04-23 2024-04-24     NaT   
164   202778        2 2023-12-28 2024-04-08 2023-11-01 2024-04-08     NaT   
165   205388        2 2024-02-23 2024-04-29 2024-05-20 2024-06-01     NaT   

    exit_3 entry_4 exit_4  ... des_3 des_4    veteran      d_o_b   househol

C:\Users\jacob\AppData\Local\Temp\ipykernel_18076\3949020089.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rep = repeat.groupby("HMIS ID").apply(process_group).reset_index(drop=True)


In [60]:
# Ensure "exit_1" and "entry_2" are datetime objects
rep["exit_1"] = pd.to_datetime(rep["exit_1"])
rep["entry_2"] = pd.to_datetime(rep["entry_2"])

# Calculate the time difference in days
rep["time_diff"] = (rep["entry_2"] - rep["exit_1"]).dt.days

# Calculate the average time difference, ignoring null values
average_time_diff = rep["time_diff"].mean()

print(f"The average time between 'exit_1' and 'entry_2' is: {average_time_diff:.2f} days.")


The average time between 'exit_1' and 'entry_2' is: 40.96 days.


In [61]:
# Create the "prog" column by combining the destination columns
rep["prog"] = rep[["des_1", "des_2", "des_3", "des_4"]].apply(
    lambda row: ", ".join(filter(None, row)), axis=1
)

# Display the resulting dataframe
print(rep)


     hmis_id  repeats    entry_1     exit_1    entry_2     exit_2 entry_3  \
0      28351        2 2021-03-30 2022-12-19 2023-07-01 2024-01-13     NaT   
1     110591        2 2023-02-22 2023-03-17 2024-01-29 2024-07-06     NaT   
2     114968        2 2022-04-29 2023-01-17 2023-05-16 2023-09-01     NaT   
3     117893        2 2023-02-10 2023-07-03 2024-04-08 2024-09-03     NaT   
4     132248        2 2022-09-23 2022-12-05 2023-07-05 2023-07-06     NaT   
..       ...      ...        ...        ...        ...        ...     ...   
161   201293        2 2023-08-31 2023-09-15 2023-09-15 2024-01-05     NaT   
162   201294        2 2023-08-31 2023-09-15 2023-09-15 2024-01-05     NaT   
163   202261        2 2023-10-12 2024-02-02 2024-04-23 2024-04-24     NaT   
164   202778        2 2023-12-28 2024-04-08 2023-11-01 2024-04-08     NaT   
165   205388        2 2024-02-23 2024-04-29 2024-05-20 2024-06-01     NaT   

    exit_3 entry_4 exit_4  ...    veteran      d_o_b   household pregnant  

In [62]:
print(rep.columns)

Index(['hmis_id', 'repeats', 'entry_1', 'exit_1', 'entry_2', 'exit_2',
       'entry_3', 'exit_3', 'entry_4', 'exit_4', 'des_1', 'des_2', 'des_3',
       'des_4', 'veteran', 'd_o_b', 'household', 'pregnant', 'chronic', 'race',
       'ethn', 'age', 'time_diff', 'prog'],
      dtype='object')


In [63]:
# Ensure "entry_1" and "exit_1" are datetime objects
rep["entry_1"] = pd.to_datetime(rep["entry_1"])
rep["exit_1"] = pd.to_datetime(rep["exit_1"])

# Calculate the time difference in days
rep["duration"] = (rep["exit_1"] - rep["entry_1"]).dt.days

# Compute the average duration
average_duration = rep["duration"].mean()

print(f"The average length of time between 'entry_1' and 'exit_1' is: {average_duration:.2f} days.")


The average length of time between 'entry_1' and 'exit_1' is: 316.72 days.


In [64]:
# Ensure "entry_1" and "exit_1" are datetime objects
rep["entry_2"] = pd.to_datetime(rep["entry_2"])
rep["exit_2"] = pd.to_datetime(rep["exit_2"])

# Calculate the time difference in days
rep["duration"] = (rep["exit_2"] - rep["entry_2"]).dt.days

# Compute the average duration
average_duration = rep["duration"].mean()

print(f"The average length of time between 'entry_2' and 'exit_2' is: {average_duration:.2f} days.")


The average length of time between 'entry_2' and 'exit_2' is: 220.07 days.


In [65]:
# Group by "prog" and count occurrences
prog = rep.groupby("prog").size().reset_index(name="count")

# Sort the resulting dataframe by "count" in descending order
prog = prog.sort_values(by="count", ascending=False).reset_index(drop=True)

# Display the resulting dataframe
print(prog)


                                                                                                                                                                                                                                                 prog  \
0                                                                                                                                                                                No exit interview completed (HUD), No exit interview completed (HUD)   
1                                                                                                                                                             No exit interview completed (HUD), Rental by client, with ongoing housing subsidy (HUD)   
2                                                                                                                                                               No exit interview completed (HUD), Rental by client, no ongoing housing subsidy (HUD)   
3   

In [66]:
# Set the maximum column width to display long strings fully
pd.set_option('display.max_colwidth', None)

# Display the dataframe
print(prog)

                                                                                                                                                                                                                                                 prog  \
0                                                                                                                                                                                No exit interview completed (HUD), No exit interview completed (HUD)   
1                                                                                                                                                             No exit interview completed (HUD), Rental by client, with ongoing housing subsidy (HUD)   
2                                                                                                                                                               No exit interview completed (HUD), Rental by client, no ongoing housing subsidy (HUD)   
3   

In [67]:
# Write the prog dataframe to a CSV file
prog.to_csv("prog_output.csv", index=False)

print("The prog dataframe has been written to prog_output.csv")


The prog dataframe has been written to prog_output.csv


In [68]:
# Combine all "des" columns into a single series
all_des = pd.concat([rep["des_1"], rep["des_2"], rep["des_3"], rep["des_4"]])

# Drop missing values
all_des = all_des.dropna()

# Get the most common values
most_common_des = all_des.value_counts()

# Display the most common values
print("Most common values in des columns:")
print(most_common_des)


Most common values in des columns:
No exit interview completed (HUD)                                                                                                      205
Rental by client, with ongoing housing subsidy (HUD)                                                                                    42
Rental by client, no ongoing housing subsidy (HUD)                                                                                      36
Jail, prison, or juvenile detention facility (HUD)                                                                                      11
Staying or living with family, temporary tenure (e.g., room, apartment, or house) (HUD)                                                  8
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, Host Home shelter (HUD)                             7
Staying or living with family, permanent tenure (HUD)                                                                              

In [69]:
# Get the most common values in "des_1"
most_common_des_1 = rep["des_1"].value_counts()

# Display the most common values
print("Most common values in des_1:")
print(most_common_des_1)


Most common values in des_1:
des_1
No exit interview completed (HUD)                                                                                                      109
Rental by client, no ongoing housing subsidy (HUD)                                                                                      15
Rental by client, with ongoing housing subsidy (HUD)                                                                                    15
Jail, prison, or juvenile detention facility (HUD)                                                                                       5
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, Host Home shelter (HUD)                             5
Staying or living with family, temporary tenure (e.g., room, apartment, or house) (HUD)                                                  4
Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside) (HUD)

In [70]:
most_common_des_2 = rep["des_2"].value_counts()

# Display the most common values
print("Most common values in des_2:")
print(most_common_des_2)


Most common values in des_2:
des_2
No exit interview completed (HUD)                                                                                                      92
Rental by client, with ongoing housing subsidy (HUD)                                                                                   24
Rental by client, no ongoing housing subsidy (HUD)                                                                                     20
Jail, prison, or juvenile detention facility (HUD)                                                                                      6
Deceased (HUD)                                                                                                                          4
Staying or living with family, permanent tenure (HUD)                                                                                   4
Psychiatric hospital or other psychiatric facility (HUD)                                                                                3

In [71]:
most_common_des_3 = rep["des_3"].value_counts()

# Display the most common values
print("Most common values in des_3:")
print(most_common_des_3)


Most common values in des_3:
des_3
No exit interview completed (HUD)                                                                               4
Rental by client, with ongoing housing subsidy (HUD)                                                            3
Staying or living with family, temporary tenure (e.g., room, apartment, or house) (HUD)                         2
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, Host Home shelter (HUD)    1
Transitional housing for homeless persons (including homeless youth) (HUD)                                      1
Rental by client, no ongoing housing subsidy (HUD)                                                              1
Name: count, dtype: int64


In [72]:
most_common_des_4 = rep["des_4"].value_counts()

# Display the most common values
print("Most common values in des_4:")
print(most_common_des_4)


Most common values in des_4:
des_4
Staying or living with friends, temporary tenure (e.g., room, apartment, or house) (HUD)    1
Name: count, dtype: int64


In [73]:
# Identify "HMIS ID" values that appear in "repeat"
repeat_ids = repeat["HMIS ID"].unique()

# Filter "dest" to exclude rows with "HMIS ID" in "repeat"
single = dest[~dest["HMIS ID"].isin(repeat_ids)].reset_index(drop=True)

# Display the resulting dataframe
print(single)


      HMIS ID  \
0      209226   
1      164401   
2      167202   
3      166440   
4      167814   
...       ...   
1560   191550   
1561   172577   
1562   203266   
1563   202076   
1564   197230   

                                                          Entry/Exit Destination  \
0                                                      Client doesn't know (HUD)   
1                                                                 Deceased (HUD)   
2                                                                 Deceased (HUD)   
3                                                                 Deceased (HUD)   
4                                                                 Deceased (HUD)   
...                                                                          ...   
1560  Transitional housing for homeless persons (including homeless youth) (HUD)   
1561  Transitional housing for homeless persons (including homeless youth) (HUD)   
1562  Transitional housing for homeless

In [74]:
# Get the most common values for "Entry/Exit Destination" in repeat
common_destinations = single["Entry/Exit Destination"].value_counts()

# Display the most common values and their counts
print("Most common 'Entry/Exit Destination' values in 'single':")
print(common_destinations)


Most common 'Entry/Exit Destination' values in 'single':
Entry/Exit Destination
No exit interview completed (HUD)                                                                                                      1094
Rental by client, with ongoing housing subsidy (HUD)                                                                                    141
Rental by client, no ongoing housing subsidy (HUD)                                                                                       99
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, Host Home shelter (HUD)                             30
Staying or living with family, permanent tenure (HUD)                                                                                    27
Deceased (HUD)                                                                                                                           25
Jail, prison, or juvenile detention facility (HUD)                              

In [75]:
row_count = len(rep)
print(f"The 'rep' dataframe contains {row_count} rows.")


The 'rep' dataframe contains 166 rows.


In [76]:
# Count rows where the column "chronic" contains "Yes (HUD)"
chronic_yes_count = (rep["chronic"] == "Yes (HUD)").sum()

print(f"The number of rows in 'rep' with 'Yes (HUD)' in the 'chronic' column is: {chronic_yes_count}")


The number of rows in 'rep' with 'Yes (HUD)' in the 'chronic' column is: 17


In [77]:
# Count rows where the column "Chronicity" contains "Yes (HUD)"
chronicity_yes_count = (single["Chronicity"] == "Yes (HUD)").sum()

print(f"The number of rows in 'single' with 'Yes (HUD)' in the 'Chronicity' column is: {chronicity_yes_count}")


The number of rows in 'single' with 'Yes (HUD)' in the 'Chronicity' column is: 94


In [78]:
print(single.columns)

Index(['HMIS ID', 'Entry/Exit Destination', 'Entry Date', 'Exit Date',
       'Days to Destination', 'Veteran', 'Date of Birth', 'Age',
       'Household Type', 'Pregnant', 'Chronicity', 'Race (Retired)',
       'Ethnicity (Retired)'],
      dtype='object')


In [79]:
# Ensure "entry_1" and "exit_1" are datetime objects
single["Entry Date"] = pd.to_datetime(single["Entry Date"])
single["Exit Date"] = pd.to_datetime(single["Exit Date"])

# Calculate the time difference in days
single["duration"] = (single["Exit Date"] - single["Entry Date"]).dt.days

# Compute the average duration
average_duration = single["duration"].mean()

print(f"The average length of time between 'Entry Date' and 'Exit Date' is: {average_duration:.2f} days.")

The average length of time between 'Entry Date' and 'Exit Date' is: 240.79 days.


In [80]:
# Ensure "Entry Date" is in datetime format
dest["Entry Date"] = pd.to_datetime(dest["Entry Date"])

# Identify rows where "HMIS ID" is not duplicated
non_duplicates = dest[~dest["HMIS ID"].duplicated(keep=False)]

# Identify the earliest row for each group of duplicate "HMIS ID" values
duplicates_earliest = (
    dest[dest["HMIS ID"].duplicated(keep=False)]  # Select duplicated rows
    .sort_values(by=["HMIS ID", "Entry Date"])    # Sort by "HMIS ID" and "Entry Date"
    .drop_duplicates(subset="HMIS ID", keep="first")  # Keep the earliest row for each "HMIS ID"
)

# Combine non-duplicates and earliest duplicates
pop = pd.concat([non_duplicates, duplicates_earliest]).reset_index(drop=True)

# Display the resulting dataframe
print(pop)


      HMIS ID                              Entry/Exit Destination Entry Date  \
0      209226                           Client doesn't know (HUD) 2024-09-12   
1      164401                                      Deceased (HUD) 2024-01-17   
2      167202                                      Deceased (HUD) 2021-10-30   
3      166440                                      Deceased (HUD) 2022-12-07   
4      167814                                      Deceased (HUD) 2023-05-12   
...       ...                                                 ...        ...   
1726   201293  Rental by client, no ongoing housing subsidy (HUD) 2023-08-31   
1727   201294  Rental by client, no ongoing housing subsidy (HUD) 2023-08-31   
1728   202261                   No exit interview completed (HUD) 2023-10-12   
1729   202778                   No exit interview completed (HUD) 2023-11-01   
1730   205388  Jail, prison, or juvenile detention facility (HUD) 2024-02-23   

      Exit Date  Days to Destination   

In [83]:
print(pop.columns)
print(rep.columns)

Index(['HMIS ID', 'Entry/Exit Destination', 'Entry Date', 'Exit Date',
       'Days to Destination', 'Veteran', 'Date of Birth', 'Age',
       'Household Type', 'Pregnant', 'Chronicity', 'Race (Retired)',
       'Ethnicity (Retired)'],
      dtype='object')
Index(['hmis_id', 'repeats', 'entry_1', 'exit_1', 'entry_2', 'exit_2',
       'entry_3', 'exit_3', 'entry_4', 'exit_4', 'des_1', 'des_2', 'des_3',
       'des_4', 'veteran', 'd_o_b', 'household', 'pregnant', 'chronic', 'race',
       'ethn', 'age', 'time_diff', 'prog', 'duration'],
      dtype='object')


In [82]:
# Calculate the average value for "Days to Destination"
average_days_to_destination = pop["Days to Destination"].mean()

print(f"The average value for 'Days to Destination' in 'pop' is: {average_days_to_destination:.2f} days.")


The average value for 'Days to Destination' in 'pop' is: 251.67 days.


In [84]:
# Count the occurrences of each unique value in the "veteran" column
veteran_counts = rep["veteran"].value_counts()

print("Number of each unique value for 'veteran' in 'rep':")
print(veteran_counts)


Number of each unique value for 'veteran' in 'rep':
veteran
No (HUD)                    137
Yes (HUD)                    21
Data not collected (HUD)      2
Name: count, dtype: int64


In [85]:
# Count the occurrences of each unique value in the "veteran" column
veteran_counts = pop["Veteran"].value_counts()

print("Number of each unique value for 'veteran' in 'pop':")
print(veteran_counts)


Number of each unique value for 'veteran' in 'pop':
Veteran
No (HUD)                              1438
Yes (HUD)                              214
Client prefers not to answer (HUD)      20
Data not collected (HUD)                11
Client doesn't know (HUD)                1
Name: count, dtype: int64
